In [20]:
#!/usr/bin/env python3
import threading
import time
import datetime
import math

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.ticktype import TickTypeEnum

###############################################################################
# Black-Scholes-Hilfsfunktionen (inkl. Rho-Berechnung)
###############################################################################

def _phi(x):
    """Dichtefunktion der Standardnormalverteilung."""
    return 1.0 / math.sqrt(2*math.pi) * math.exp(-0.5 * x * x)

def _Phi(x):
    """Verteilungsfunktion (CDF) der Standardnormalverteilung."""
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def black_scholes_call(S, K, T, r, sigma, q=0.0):
    """Preis einer Call-Option nach Black-Scholes."""
    if T <= 0:
        return max(0, S - K)
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * math.exp(-q * T) * _Phi(d1) - K * math.exp(-r * T) * _Phi(d2)

def black_scholes_put(S, K, T, r, sigma, q=0.0):
    """Preis einer Put-Option nach Black-Scholes."""
    if T <= 0:
        return max(0, K - S)
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return K * math.exp(-r * T) * _Phi(-d2) - S * math.exp(-q * T) * _Phi(-d1)

def black_scholes_rho(S, K, T, r, sigma, q=0.0, is_call=True):
    """Berechnet Rho einer Option (Call/Put)."""
    if T <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    # Rho = Ableitung des Optionspreises nach r
    # Für Calls:  K * T * exp(-r*T) * Phi(d2)
    # Für Puts:  -K * T * exp(-r*T) * Phi(-d2)
    if is_call:
        return K * T * math.exp(-r*T) * _Phi(d2)
    else:
        return -K * T * math.exp(-r*T) * _Phi(-d2)

def vega(S, K, T, r, sigma, q=0.0, is_call=True):
    """Vega = d(Preis)/d(Vol)."""
    if T <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return S * math.exp(-q*T) * math.sqrt(T) * _phi(d1)

def implied_vol_newton(option_price, S, K, T, r, q=0.0, is_call=True,
                       max_iter=100, tol=1e-6):
    """
    Implizite Volatilität via Newton-Raphson.
    option_price: Marktpreis (z.B. (Bid+Ask)/2)
    S: Underlying-Kurs
    K: Strike
    T: (Verfalls - Heute) in Jahren
    r: risikofreier Zins
    q: Dividendenrendite (vereinfacht 0)
    is_call: True = Call, False = Put
    """
    if option_price < 0.01:
        return None
    sigma = 0.20  # Startwert 20%
    for i in range(max_iter):
        if is_call:
            price_est = black_scholes_call(S, K, T, r, sigma, q)
        else:
            price_est = black_scholes_put(S, K, T, r, sigma, q)
        diff = price_est - option_price
        if abs(diff) < tol:
            return sigma
        v = vega(S, K, T, r, sigma, q, is_call)
        if v < 1e-10:
            return None
        sigma = sigma - diff / v
        if sigma < 0:
            sigma = 0
    return None

###############################################################################
# Hauptklasse
###############################################################################
class OptionChainApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

        self.option_contracts = []     # Bis zu 10 Contracts (nur Calls)
        self.reqId_to_contract = {}    # reqId -> Contract
        self.option_values = {}        # Alle Daten (Bid, Ask, Last, IV, Greeks etc.)

        self.contract_details_received = False
        self.mkt_data_count = 0
        self.mkt_data_done = False

    def error(self, reqId, errorCode, errorString, *rest):
        print(f"[ERROR] reqId={reqId}, code={errorCode}, msg={errorString}")

    def contractDetails(self, reqId, contractDetails):
        """Sammelt bis zu 10 Call-Optionen zu einem Underlying."""
        c = contractDetails.contract
        # Maximal 10, nur Calls
        if len(self.option_contracts) < 10:
            if c.secType == "OPT" and c.right == "C":
                self.option_contracts.append(c)
                print(f"[INFO] Option gefunden: {c.symbol} {c.lastTradeDateOrContractMonth} "
                      f"{c.right} {c.strike}")
        else:
            pass  # keine weiteren sammeln

    def contractDetailsEnd(self, reqId):
        self.contract_details_received = True
        print(f"[INFO] Ende contractDetails (insg. {len(self.option_contracts)} Optionen).")

    def tickPrice(self, reqId, tickType, price, attrib):
        if reqId not in self.reqId_to_contract:
            return
        contract = self.reqId_to_contract[reqId]
        key = self._contract_key(contract)

        if tickType == TickTypeEnum.BID:
            self.option_values[f"{key}-bid"] = price
        elif tickType == TickTypeEnum.ASK:
            self.option_values[f"{key}-ask"] = price
        elif tickType == TickTypeEnum.LAST:
            self.option_values[f"{key}-last"] = price

    def tickSize(self, reqId, tickType, size):
        if reqId not in self.reqId_to_contract:
            return
        contract = self.reqId_to_contract[reqId]
        key = self._contract_key(contract)

        if tickType == TickTypeEnum.BID_SIZE:
            self.option_values[f"{key}-bidSize"] = size
        elif tickType == TickTypeEnum.ASK_SIZE:
            self.option_values[f"{key}-askSize"] = size
        elif tickType == TickTypeEnum.LAST_SIZE:
            self.option_values[f"{key}-lastSize"] = size
        elif tickType == TickTypeEnum.VOLUME:
            self.option_values[f"{key}-volume"] = size

    def tickOptionComputation(self, reqId, tickType, impliedVol, delta, optPrice, pvDividend,
                              gamma, vega_, theta, undPrice, _):
        """
        Greeks + Underlying-Price empfangen.
        Wir ignorieren impliedVol von IB und berechnen selbst.
        """
        if reqId not in self.reqId_to_contract:
            return
        contract = self.reqId_to_contract[reqId]
        key = self._contract_key(contract)

        # Kurs des Underlyings
        if undPrice and undPrice > 0:
            self.option_values[f"{key}-undPrice"] = undPrice

        # Greeks von IB (falls != -1)
        if delta not in (None, -1):
            self.option_values[f"{key}-delta"] = round(delta, 4)
        if gamma not in (None, -1):
            self.option_values[f"{key}-gamma"] = round(gamma, 4)
        if vega_ not in (None, -1):
            self.option_values[f"{key}-vega"] = round(vega_, 4)
        if theta not in (None, -1):
            self.option_values[f"{key}-theta"] = round(theta, 4)

    def _contract_key(self, c):
        """Eindeutiger Key für ein Contract-Objekt."""
        strike_str = f"{c.strike:08.2f}".replace('.', '')
        return f"{c.symbol}_{c.lastTradeDateOrContractMonth}_{c.right}_{strike_str}"

    def start_data_requests(self):
        """Verzögerte Market-Data anfordern (snapshot=True)."""
        self.reqMarketDataType(3)  # 3 = Delayed data
        for i, c in enumerate(self.option_contracts):
            req_id = 1000 + i
            self.reqId_to_contract[req_id] = c
            self.reqMktData(req_id, c, "", True, False, [])
            self.mkt_data_count += 1

    def stop_data_requests(self):
        for req_id in list(self.reqId_to_contract.keys()):
            self.cancelMktData(req_id)
        print("[INFO] MktData Requests beendet.")
        self.mkt_data_done = True

###############################################################################
# Hauptprogramm
###############################################################################
def main():
    app = OptionChainApp()
    app.connect("127.0.0.1", 7497, clientId=100)

    thread = threading.Thread(target=app.run, daemon=True)
    thread.start()

    time.sleep(1)

    # 1) ContractDetails für AAPL-Optionen anfordern
    c = Contract()
    c.symbol = "AAPL"
    c.secType = "OPT"
    c.exchange = "SMART"
    c.currency = "USD"

    app.reqContractDetails(1, c)

    # Warten auf Ende contractDetails
    while not app.contract_details_received:
        time.sleep(0.2)

    print(f"[INFO] Gesammelte Optionen: {len(app.option_contracts)}")

    # 2) Delayed Market-Data anfordern (snapshot)
    app.start_data_requests()

    # Warten, bis Daten ankommen (snapshot kann 2-5 Sekunden dauern)
    time.sleep(8)

    # 3) MktData stoppen
    app.stop_data_requests()
    time.sleep(1)

    # Verbindung trennen
    app.disconnect()
    thread.join()

    # 4) Nun manuell IV & Rho berechnen
    #    (wir haben bid/ask/last, gamma, delta, vega, undPrice)
    results = []
    r = 0.05   # 5% Zins
    q = 0.0    # 0% Div-Rendite

    for c in app.option_contracts:
        key = app._contract_key(c)
        # Eckdaten aus option_values
        bid = app.option_values.get(f"{key}-bid", None)
        ask = app.option_values.get(f"{key}-ask", None)
        last = app.option_values.get(f"{key}-last", None)
        undPrice = app.option_values.get(f"{key}-undPrice", None)

        # Greeks (teilweise von IB, Rho manuell)
        delta = app.option_values.get(f"{key}-delta", None)
        gamma = app.option_values.get(f"{key}-gamma", None)
        vega = app.option_values.get(f"{key}-vega", None)
        theta = app.option_values.get(f"{key}-theta", None)

        # Market price -> (bid+ask)/2 oder fallback last
        market_price = None
        if bid is not None and ask is not None and bid>0 and ask>0:
            market_price = (bid + ask)/2
        elif last is not None and last>0:
            market_price = last

        # Zeit bis Verfall T (in Jahren)
        try:
            expiry_date = datetime.datetime.strptime(c.lastTradeDateOrContractMonth, "%Y%m%d")
            T = (expiry_date - datetime.datetime.now()).days / 365.0
            if T < 0: T = 0
        except:
            T = 0

        # Implizite Vol selbst rechnen
        iv_calc = None
        rho_calc = None
        if (market_price and undPrice and undPrice>0 and T>0):
            iv_calc = implied_vol_newton(
                option_price=market_price,
                S=undPrice,
                K=c.strike,
                T=T,
                r=r,
                q=q,
                is_call=(c.right=="C")
            )
            # Falls IV konvergiert, berechne Rho
            if iv_calc:
                rho_calc = black_scholes_rho(
                    S=undPrice, K=c.strike, T=T, r=r,
                    sigma=iv_calc, q=q, is_call=(c.right=="C")
                )

        results.append({
            "symbol": c.symbol,
            "expiry": c.lastTradeDateOrContractMonth,
            "type": c.right,
            "strike": c.strike,
            "bid": bid,
            "ask": ask,
            "last": last,
            "undPrice": undPrice,
            "delta": delta,
            "gamma": gamma,
            "vega": vega,
            "theta": theta,
            "iv_calc": round(iv_calc*100, 2) if iv_calc else None,
            "rho_calc": round(rho_calc, 4) if rho_calc else None,
        })

    # 5) Ausgabe aller gesammelten Daten (die 10 Optionen)  
    print("\n================= ERGEBNISSE (10 Option-Kontrakte) =================")
    for r_ in results:
        print(r_)
    print("====================================================================")

if __name__ == "__main__":
    main()

[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:hfarm
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm.nj
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:jfarm
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:usfuture
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:cashfarm
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:usopt
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:eufarmnj
[ERROR] reqId=-1, code=2104, msg=Market data farm connection is OK:usfarm
[ERROR] reqId=-1, code=2106, msg=HMDS data farm connection is OK:euhmds
[ERROR] reqId=-1, code=2106, msg=HMDS data farm connection is OK:fundfarm
[ERROR] reqId=-1, code=2106, msg=HMDS data farm connection is OK:ushmds
[ERROR] reqId=-1, code=2158, msg=Sec-def data farm connection is OK:secdefil
[INFO] Option gefunden: AAPL 20250620 C 105.0
[INFO] Option gefunden: AAPL 20250620 C 115.0
[INFO] Option g

In [1]:
from ibapi.ticktype import TickType, TickTypeEnum
import yfinance

In [10]:
TickTypeEnum.toStr(4)

'LAST'